In [7]:
import numpy as np
import pandas as pd

In [8]:
df=pd.read_csv(r'C:\Users\Lenovo\Downloads\data set\creditcard.csv')

In [9]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [10]:
df=df.iloc[:,1:]

In [11]:
df.shape

(284807, 30)

In [12]:
df['Class'].value_counts()  

Class
0    284315
1       492
Name: count, dtype: int64

In [13]:
X=df.drop('Class',axis=1)

In [14]:
y=df['Class']

In [15]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()

In [16]:
from sklearn.model_selection import train_test_split

# Reduce dataset to 30% of original size, preserving class distribution
X_reduced, _, y_reduced, _ = train_test_split(
    X, y, 
    test_size=0.7, 
    stratify=y,  # Target variable for stratification
    random_state=42
)


In [17]:
X_reduced.shape[0]*100/df.shape[0]

29.999964888503442

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y_reduced, test_size=0.2, random_state=42)

In [19]:
X_train=sc.fit_transform(X_train)

In [20]:
X_test=sc.transform(X_test)

In [21]:
from imblearn.over_sampling import SMOTE

In [22]:
# Apply SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)


In [23]:
y_train_smote.value_counts()

Class
0    68232
1    68232
Name: count, dtype: int64

In [24]:
from sklearn.ensemble import RandomForestClassifier

In [25]:
RCF=RandomForestClassifier(n_jobs=-1,random_state=42)

In [26]:
RCF.fit(X_train_smote,y_train_smote)

RandomForestClassifier(n_jobs=-1, random_state=42)

In [27]:
from sklearn.metrics import accuracy_score,classification_report,f1_score,confusion_matrix

In [28]:
y_pred=RCF.predict(X_test)

In [29]:
accuracy_score(y_test,y_pred)

0.9995318626016736

In [30]:
f1_score(y_test,y_pred) 

0.8518518518518519

In [31]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     17062
           1       0.85      0.85      0.85        27

    accuracy                           1.00     17089
   macro avg       0.93      0.93      0.93     17089
weighted avg       1.00      1.00      1.00     17089



In [32]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

In [33]:
param_grid = {
    'criterion': ['gini', 'entropy','log_loss'],
    'n_estimators': [100, 150],
    'max_depth': [5, 15],
    'bootstrap': [True]
}


In [46]:
skf = StratifiedKFold(n_splits=4,shuffle=True, random_state=42)

In [35]:
random_search = RandomizedSearchCV(estimator=RCF,param_distributions=param_grid, verbose=2, n_jobs=-1,cv=skf,random_state=42,scoring='f1')


In [36]:
random_search.fit(X_train_smote, y_train_smote)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=42, shuffle=True),
                   estimator=RandomForestClassifier(n_jobs=-1, random_state=42),
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True],
                                        'criterion': ['gini', 'entropy',
                                                      'log_loss'],
                                        'max_depth': [5, 15],
                                        'n_estimators': [100, 150]},
                   random_state=42, scoring='f1', verbose=2)

In [37]:
print("Best Parameters: ", random_search.best_params_)

# Make predictions using the best model
y_pred = random_search.best_estimator_.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Best Parameters:  {'n_estimators': 150, 'max_depth': 15, 'criterion': 'log_loss', 'bootstrap': True}
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     17062
           1       0.80      0.89      0.84        27

    accuracy                           1.00     17089
   macro avg       0.90      0.94      0.92     17089
weighted avg       1.00      1.00      1.00     17089

[[17056     6]
 [    3    24]]


In [47]:
import numpy as np

# Get mean test scores for F1-score during cross-validation
mean_f1_scores = random_search.cv_results_['mean_test_score']

# Compute the overall mean of F1-scores across all tested parameter sets
mean_f1_score = np.mean(mean_f1_scores)

print(f"Mean F1-Score from RandomizedSearchCV: {mean_f1_score:.4f}")


Mean F1-Score from RandomizedSearchCV: 0.9811
